# 第4回講義 宿題

## 課題
今Lessonで学んだことを元に, MNISTのファッション版 (Fashion MNIST, クラス数10) を多層パーセプトロン (MLP) によって分類してみましょう.

Fashion MNISTの詳細については以下のリンクを参考にしてください.

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 85%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **MLPのアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap04`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap04 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('../data/x_train.npy')
    y_train = np.load('../data/y_train.npy')
    
    # テストデータ
    x_test = np.load('../data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### 多層パーセプトロンの実装

In [5]:
%%writefile submission_code.py

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
np.random.seed(7)

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

def np_log(x):
    # WRITE ME
    return np.log(np.clip(x, 1e-10, x))

def relu(x):
    # WRITE ME
    return np.maximum(x, 0)

def deriv_relu(x):
    # WRITE ME
    return (x > 0).astype(x.dtype)

def softmax(x):
    # WRITE ME
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    # WRITE ME
    return softmax() * (1 - softmax(x))

class Dense:
    # WRITE ME
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low=-0.08, high=0.08,
                                   size=(in_dim, out_dim)).astype('float64')
        self.b = np.zeros(out_dim).astype('float64')
        self.function = function
        self.deriv_function = deriv_function
        
        self.x = None
        self.u = None
        
        self.dW = None
        self.db = None

        self.params_idxs = np.cumsum([self.W.size, self.b.size])

    def __call__(self, x):
        self.x = x
        self.u = np.matmul(self.x, self.W) + self.b
        return self.function(self.u)

    def b_prop(self, delta, W):
        self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T)
        return self.delta
    
    def compute_grad(self):
        batch_size = self.delta.shape[0]
        
        self.dW = np.matmul(self.x.T, self.delta) / batch_size
        self.db = np.matmul(np.ones(batch_size), self.delta) / batch_size

    def get_params(self):
        return np.concatenate([self.W.ravel(), self.b], axis=0)
    
    def set_params(self, params):
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b
    
    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis=0)

def f_props(layers, x):
    # WRITE ME
    for layer in layers:
        x = layer(x)
    return x

def b_props(layers, delta):
    # WRITE ME
    batch_size = delta.shape[0]
    
    for i, layer in enumerate(layers[::-1]):
        if i == 0: # 出力層の場合
            layer.delta = delta # y - t
            layer.compute_grad() # 勾配の計算
        else: # 出力層以外の場合
            delta = layer.b_prop(delta, W) # 逆伝播
            layer.compute_grad() # 勾配の計算

        W = layer.W

def update_parameters(layers, eps):
    # WRITE ME
    for layer in layers:
        layer.W -= eps * layer.dW

        layer.b -= eps * layer.db

# WRITE ME
layers = [
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

def train_mnist(x, t, eps):
    # WRITE ME
    # 順伝播
    y = f_props(layers, x)

    # 誤差の計算
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    # 逆伝播
    delta = y - t
    b_props(layers, delta)

    # パラメータの更新
    update_parameters(layers, eps)

    return cost

def valid_mnist(x, t):
    # WRITE ME
    # 順伝播
    y = f_props(layers, x)
    
    # 誤差の計算
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    return cost, y

for epoch in range(5):
    x_train, y_train = shuffle(x_train, y_train)
    # オンライン学習
    # WRITE ME
    for x, t in zip(x_train, y_train):
        cost = train_mnist(x[None, :], t[None, :], eps=0.01)
    
    cost, y_pred = valid_mnist(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(epoch + 1, cost, accuracy))

# y_pred = np.argmax(y_pred, axis=1)
y_test = y = f_props(layers, x_test)
y_test = y_test.argmax(axis=1)

submission = pd.Series(y_test, name='label')
submission.to_csv('submission_pred.csv', header=True, index_label='id')

Overwriting submission_code.py


In [3]:
print(y_pred.argmax(axis=1))
print(y_valid.argmax(axis=1))

[0 1 4 2 5 1 8 5 4 8 2 3 9 2 2 8 9 5 7 9 7 6 6 8 6 1 6 9 7 0 0 5 3 8 6 9 7
 9 9 9 5 4 3 8 3 9 5 2 1 1 0 9 1 2 5 0 6 9 4 7 4 0 4 5 1 5 4 2 2 0 7 7 5 1
 4 0 4 7 0 9 9 6 7 1 2 3 6 0 6 3 3 2 6 7 4 8 9 0 5 5 1 4 5 5 0 6 0 5 8 9 4
 9 1 6 9 1 2 6 2 7 5 7 9 0 3 0 1 5 3 6 4 0 3 9 9 3 4 3 0 4 9 8 2 0 7 4 8 7
 1 1 1 5 3 9 2 0 9 6 9 5 2 3 6 2 7 8 7 7 9 0 1 3 2 6 4 4 7 7 0 1 9 4 4 7 4
 3 3 2 5 8 0 6 3 0 5 0 0 8 2 9 5 8 9 6 1 1 2 4 8 9 8 1 4 8 4 4 4 7 6 9 3 1
 1 9 5 7 2 7 0 7 2 4 7 1 6 4 2 2 7 7 9 8 8 2 7 4 9 4 9 8 9 8 9 1 0 8 9 5 0
 6 7 8 3 8 9 4 2 1 8 4 6 5 4 5 6 7 1 9 6 3 4 8 8 1 9 5 4 7 0 7 7 9 8 9 9 7
 6 4 8 0 7 8 4 6 3 0 4 0 4 5 8 7 7 3 5 7 3 9 6 8 4 0 8 4 8 8 5 8 8 5 5 6 8
 8 1 0 8 2 1 1 0 7 8 8 1 5 5 1 9 4 0 3 8 3 0 9 2 7 1 4 1 9 5 9 9 0 4 5 0 0
 9 4 1 2 8 9 6 8 6 0 1 9 3 0 0 0 9 0 7 0 1 4 9 3 3 8 2 3 2 8 3 6 1 4 1 1 5
 3 7 8 2 9 6 1 4 8 0 3 0 9 2 8 9 5 1 8 7 1 7 1 5 0 0 5 6 1 9 8 3 1 2 2 0 1
 8 9 4 3 1 6 7 0 4 7 3 8 1 1 1 8 3 3 3 7 7 9 5 5 1 0 4 4 8 9 6 5 1 4 6 8 7
 0 5 4 8 7 5 8 2 0 7 4 9 

In [4]:
print(y_test)

[4 4 6 ... 6 9 6]
